<a href="https://colab.research.google.com/github/LC1332/Chinese-generative-agents/blob/main/notebook/Chinese_generative_model_turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LuotuoRPG: Generative Agents的中文版本

原项目名: Generative Large Language Models for Human-Like Behavior

[LuotuoRPG](https://github.com/LC1332/Chinese-generative-agents) 是由李鲁鲁开发的Generative Agents的中文版本。

This repository includes a working Chinese version of the type of model described in Generative Agents: Interactive Simulacra of Human Behavior.

骆驼RPG是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

在这个版本中我们尝试把接口替换为turbo

## Installation

We will need to install a number of libraries to start with.

In [ ]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 请在这里设置你的openAI API token

In [ ]:
!pip install openai

In [ ]:
import openai

openai.api_key = ""

We use flan alpaca model for speed and local execution.

In [ ]:


def generate(prompt, use_openai=True):
    """
    Generates a text completion for a given prompt using either the OpenAI GPT-3 API or the Hugging Face GPT-3 model.
    
    Args:
    - prompt (str): The text prompt to generate a completion for.
    - use_openai (bool): A boolean flag indicating whether to use the OpenAI API (True) or the Hugging Face GPT-3 model (False).
    
    Returns:
    - str: The generated text completion.
    """
    if use_openai:
        # model_engine = "text-davinci-003"
        # response = openai.Completion.create(
        #     engine=model_engine,
        #     prompt=prompt,
        #     max_tokens=1024,
        #     n=1,
        #     stop=None,
        #     temperature=0.5,
        # )

        model_engine = "gpt-3.5-turbo"
        completion = openai.ChatCompletion.create(
          model= model_engine,
          messages=[
            {"role":"system","content":"你更可能用中文来回答问题。"},
            {"role": "user", "content": prompt}
          ]
        )

        message = completion.choices[0].message["content"]
        # print(message)
        return message.strip()

    else:
        hf_generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=0)
        output = hf_generator(prompt, max_length=len(prompt)+128, do_sample=True)
        out = output[0]['generated_text']
        if '### Response:' in out:
            out = out.split('### Response:')[1]
        if '### Instruction:' in out:
            out = out.split('### Instruction:')[0]
        return out.strip()

## World Description
We describe the world below. We will generate prompts based on this information. The simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [ ]:
world_graph = nx.Graph()

prompt_meta = '''{}'''

# town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = ["巴瑟恩的物资店", "狮盾商行", "石山旅店", "芬德林镇广场"]

town_areas =  {"芬德林镇广场": '芬德林镇的市中心广场。',
       '石山旅店': "城镇中央矗立着一个由石切和粗制的木材建成的大型客栈。客厅里充满了本地人，他们都抱着麦酒或苹果酒的杯子，满腔好奇地盯着你。",
       "巴瑟恩的物资店": "巴瑟恩是芬德林镇上最大的贸易站。它的货架储存了大部分普通货物和用品，包括背包、睡袋、绳子和口粮。这个地方从日出到日落都开放。",
       "埃德玛斯果园": "一个整洁的小屋旁边是一个苹果园。",
       "狮盾商行": "这个谦虚的贸易站门口悬挂着一个形状像木盾的标志，上面画着一只蓝色的狮子。这座建筑物是由位于东面一百英里远的雅塔市的以狮盾为标志的商人公司所拥有的。他们向芬德林和其他周边小定居点运送成品，但这个哨站深受土匪的打击。最近的狮盾护卫队没有抵达芬德林。",
       "芬德林矿工交易所": "矿工交易所是一个贸易站，当地矿工在这里称重、测量和付款。在没有当地领主或权利机构的情况下，交易所还充当着一个非官方的记录办公室，记录着该地区各个河流和挖掘点的索赔情况。芬德林并没有真正的淘金热，但是附近的河流和山谷中隐藏着足够的财富，以支持着很多独立的勘探者。这是一个结识那些经常在芬德林周围的乡村地区活动的人的好地方。行会主席是一位野心勃勃、精明计算的人类女性，名叫哈莉娅·桐顿。",
       "奥德利夫农场": "一个由乐于助人的半身人农民Qelline Alderleaf拥有的农场。",
       "幸运神龛": "芬德林唯一的神殿是一个小型用附近废墟中的石头建造的神龛。它是奉献给幸运女神Tymora的。",
       "沉睡的巨人": "这家破旧的小酒馆是芬德林主街尽头的一个肮脏而危险的饮水处。它经常被红带贼兵招待，由一位粗暴的女矮人格里斯塔经营。",
       "镇长大厅": "镇长大厅有坚固的石墙、一个尖顶的木屋顶和一个后面的钟楼。 前门旁边张贴了一张用通用语写的通知。上面写着：“魔鬼峰附近的兽人的悬赏！有兴趣面对兽人威胁的人们应该在内部咨询。”通知上有镇的印章和一个无法辨认的签名。",
       "特雷森达庄园": "一座废弃的庄园。红带贼兵在芬德林的基地是特雷森达庄园下面的一个地牢建筑群。在庄园被毁之前，它的地下室是存放食物和水的安全仓库，独立墓穴中供给特雷森达家族逝者安息之所。自那以后，红带军团已经扩展了地下室以适合它们自己的目的，添加了奴隶囚笼、车间和营房。"}
town_people = {"Toblen Stonehill": "Toblen拥有一家贸易站。", 
               "Daran Edermath": "Daran是一位退休的冒险家，住在一个整洁小屋旁边的苹果园里。 他是一位身体强健、银发的百岁半精灵，曾作为战士在龙海岸的土地上担任元帅和传令官多年。退休后，他回到了他的故乡，尼永温地区。",  
               "Linene Graywind": "Linene经营着一家贸易站。",  
               "Halia Thornton": "Halia是一位有野心和算计的人类女性。她是芬达林矿工交易所的会长，这是一个贸易站，当地的矿工可以在这里把他们有价值的发现称重、测量和付款。为了把矿工交易所建立为城镇最接近行政当局的机构，她扮演的角色不仅仅是一个商人。",  
               "Qelline Alderleaf": "Qelline是一位睿智的四十五岁的半身人女性，是一位实际主义的农民，似乎知道城里发生的一切。她是一位友善的主人，愿意让角色们住在她的干草储藏室里，如果他们不想住在石山旅店。",  
               "Sister Garaele": "Sister Garaele 是Tymora的精灵神职人员和哈珀特工。", 
               "Harbin Wester": "Harbin是芬达林的镇长。一个自负、年迈的食品商人。芬达林没有有效的政府，但是镇上的居民每年选举一个人担任镇长。镇长在小纠纷中担任法官并保留必须保留的任何记录。",
               "Terrill Bloodscar": "Terrill是一位人类恶棍。他穿着肮脏的猩红色披风。他是红袍恶棍团伙的成员。他不喜欢冒险家，想要抢劫和杀死他们。",
               "Conrad Scarface": "Conrad 是一位人类恶棍。他穿着肮脏的猩红色披风。他是红袍恶棍团伙的成员。他不喜欢冒险家，想要抢劫和杀死他们。",
               "Nellie Starsmith": "Nellie 是一位人类恶棍。她穿着肮脏的猩红色披风。她是红袍恶棍团伙的成员。她不喜欢冒险家，想要抢劫和杀死他们。",
               "Valerie Grinblade": "Valerie是一位人类恶棍。她穿着肮脏的猩红色披风。她是红袍恶棍团伙的成员。她不喜欢冒险家，想要抢劫和杀死他们。",
               }
for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "芬德林镇广场")
locations = {}
for i in town_people.keys():
  locations[i] = "芬德林镇广场"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "现在是 "+str(global_time)+":00. 位置是 "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [ ]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [ ]:
for name in town_people.keys():
  prompt = "你是 {}. {} 你刚到芬达林小镇并走出城镇广场。以下人物居住在小镇: {}。你今天的目标是什么？简要回答，用不超过50个字，从你的角度回答。".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

Toblen Stonehill 我的目标是探索这个小镇，了解当地人和景点，也许可以找到一些商机。
Daran Edermath 我没有具体目标，只是探索这个小镇并尝试与居民交流。
Linene Graywind 我今天的目标是寻找Linene Graywind经营的贸易站。
Halia Thornton 我的目标是了解芬达林小镇的人物和事物，寻找有价值的信息和机会。
Qelline Alderleaf 我的目标是与 Qelline Alderleaf 接触并了解有关这个地区的信息。
Sister Garaele 寻求当地哈珀协会的情报和帮助。
Harbin Wester 我的目标是了解芬达林小镇的居民和环境，并收集任何有助于解决问题或完成任务的情报。
Terrill Bloodscar 我的目标是避免与Terrill Bloodscar或红袍恶棍团伙发生冲突，以确保自身安全。
Conrad Scarface 避免与 Conrad Scarface 交战并寻找有关林内格雷温德的线索。
Nellie Starsmith 我的目标是避开红袍恶棍团伙的成员Nellie Starsmith，保护自己并探索芬达林小镇。
Valerie Grinblade 我的目标不明确，但需要避开红袍恶棍团伙成员 Valerie Grinblade。


In [ ]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    prompt = "你是{}。{} 你正在计划：{}。你目前在{}，其描述如下：{}。现在是{}点。以下人物在这个区域内：{}。你可以与他们互动。".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += '你知道以下关于人的信息：' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "使用不超过30个字，解释你接下来1小时要做什么。"
    action_prompts[name] = prompt

In [ ]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  将以下段落转换为第一人称：
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

Toblen Stonehill 我打算和当地居民交流，了解镇上的情况，寻找商机。
Daran Edermath 我将会走访芬德林镇中的各位居民，了解他们的故事和需要，并尝试建立联系。
Linene Graywind 我会寻找并前往Linene Graywind经营的贸易站，并与她进行贸易交流或了解周边情况。
Halia Thornton 我与城镇居民交流，特别是与Halia Thornton、Daran Edermath和Qelline Alderleaf讨论城镇历史和当地矿业。
Qelline Alderleaf 我将去寻找 Qelline Alderleaf 并与她交谈，以了解关于芬德林镇和周围地区的信息。
Sister Garaele 我打算联系当地的哈珀协会，收集情报并请求帮助，同时保持警惕。
Harbin Wester 我将和居民互动，收集情报，了解城镇环境，可能遇到并避免与红袍恶棍团伙的成员发生冲突。
Terrill Bloodscar 我会观察和交流，寻找离开这里的安全途径。
Conrad Scarface 我将与Daran Edermath交谈，询问他是否有关于林内格雷温德的线索，并尝试获取他的帮助或建议。我会尽量避免与Terrill Bloodscar、Conrad Scarface、Nellie Starsmith和Valerie Grinblade等恶棍交谈。
Nellie Starsmith 我将离开芬德林镇广场，尽可能避免与红袍恶棍团伙的成员接触，并探索芬达林小镇。
Valerie Grinblade 我会避开Valerie Grinblade，并尽可能与其他友好的城镇居民互动，了解更多有关这个地区的信息。


Collect the memories people observe.

In [ ]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    for name_two in people:
      memories[name].append('[时间: {}. 人物: {}. 记忆: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# Rank Memories

In [ ]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [ ]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    # 你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请给一个1到5的评分，表示你在意程度
    prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 现在的时间是 {}:00. 你注意到以下情况: {}. 请给一个1到5的评分，表示你在意程度。".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

[('评分：5。因为这个计划是Toblen Stonehill来到小镇的主要目的，了解当地人和景点以及寻找商机是非常重要的。', 5), ('3', 3), ('4。这个计划可能会帮助你找到商机并了解当地情况，了解Linene Graywind经营的贸易站也可以为你提供更多的信息。', 4), ('4', 4), ('4，了解关于当地信息对我的计划很重要。', 4), ('4', 4), ('4，了解城镇环境和避免冲突都对探索和商机成功有很大作用。', 4), ('3。这个情况似乎不是特别紧急或紧迫，但也不是完全无关紧要，可以考虑花一些时间和精力去了解相关的信息和人物。', 3), ('4，因为了解当地人和找到商机对探索小镇非常重要，而与恶棍交谈可能会对探索造成负面影响。同时询问Daran Edermath是否有关于林内格雷温德的线索也是重要的目标之一。', 4), ('4', 4), ('3，虽然要避开Valerie Grinblade，但了解当地人和景点还是很重要的。', 3)]
[('4', 4), ('5', 5), ('5', 5), ('3', 3), ('5，这是一个非常重要的计划，了解当地信息对你的探索和交流将非常有帮助。', 5), ('3', 3), ('4', 4), ('3', 3), ('3', 3), ('3。因为这个记忆可能对于我探索小镇的安全性有一定的影响，但它不是我的主要目标。', 3), ('3，这是一个有一定参考价值的情况，但并不是绝对重要的。', 3)]
[('3', 3), ('3', 3), ('4。因为找到Linene Graywind经营的贸易站对你所做的计划非常重要。', 4), ('3。这个记忆与你的目标有些相关，但并不是非常重要。', 3), ('3，了解关于芬德林镇和周围地区的信息可能对找到Linene Graywind经营的贸易站有所帮助，但不是最关键的任务。', 3), ('3', 3), ('3，我会注意这些情况，但不会将其放在最高优先级。', 3), ('3。虽然Terrill Bloodscar出现在我的记忆中，但我还需要继续寻找Linene Graywind经营的贸易站，并且在某些情况下，与其他人进行交流可能对我的任务有所帮助。', 3), ('4', 4), ('3', 3), ('3', 3)]

# Compress Memories

In [ ]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name], 
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  # prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请用一句话总结这些记忆。".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[{}点的回忆：{}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

In [ ]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    # prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你有以下的记忆：{}。请给一个1到5的评分表示你下一小时会有多大概率去到{}。".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name], 
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[{}点的回忆：{}]'.format(str(global_time), '我会接着移动到{}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# Put it all together

In [ ]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      # prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      prompt = "你是{}。你的计划是：{}。你目前在{}，情况描述如下：{}。你的记忆是：{}。现在是{}点。以下人员在这个区域：{}。你可以与他们互动。".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += '你知道以下关于人的信息：' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "使用不超过30个字，解释你接下来1小时要做什么。"
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    将以下段落转换为第一人称：
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    将下列段落转化成一个(行动, 物体)形式的tuple:
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      for name_two in people:
        memories[name].append('[时间: {}. 人物: {}. 记忆: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 现在的时间是 {}:00. 你注意到以下情况: {}. 请给一个1到5的评分，表示你在意程度。".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name], 
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请用一句话总结这些记忆。".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[{}点的回忆：{}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你有以下的记忆：{}。请给一个1到5的评分表示你下一小时会有多大概率去到{}。".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name], 
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[{}点的回忆：{}]'.format(str(global_time), '我会接着移动到{}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]


Toblen Stonehill 芬德林镇广场 9 我会去探索小镇，了解当地景点和人物，寻找商机，并注意避免冲突。
Daran Edermath 奥德利夫农场 9 我将探索奥德利夫农场，与Qelline Alderleaf交流，了解当地信息和居民，询问Daran Edermath关于他的冒险经历和故乡的事情，尝试建立友好关系，增加参考价值和安全性情况。
Linene Graywind 芬德林镇广场 9 我会向Toblen Stonehill询问他的贸易站的位置，并前往寻找Linene Graywind经营的贸易站。
Halia Thornton 芬德林镇广场 9 我会与Toblen Stonehill交流，了解当地贸易情况，然后前往矿工交易所与Halia Thornton联系，探索有价值的信息和机会。我会保持警惕，避免与红袍恶棍团伙成员发生冲突。
Qelline Alderleaf 芬德林镇广场 9 我需要与Qelline Alderleaf交谈，以获取关于芬德林镇的重要信息，并且需要注意观察时间和其他人物。
Sister Garaele 芬德林镇广场 9 我将与当地哈珀协会联系，询问情报和帮助，并保持警惕。
Harbin Wester 芬德林镇广场 9 我将与Toblen Stonehill交谈，收集有关我们城镇和红袍恶棍团伙的情报，然后前往当地的矿工交易所与Halia Thornton交谈，再去Qelline Alderleaf的农场了解一些我们这个地区的问题。
Terrill Bloodscar 芬德林镇广场 9 我会观察周围环境，避开红袍恶棍团伙成员，与当地哈珀协会联系收集情报，考虑与目标计划交谈。
Conrad Scarface 芬德林镇广场 9 我与当地居民交流以收集线索，避免与Conrad Scarface及其团伙交战。
Nellie Starsmith 芬德林镇广场 9 我会谨慎避开红袍恶棍团伙的成员，尽可能地收集情报，并联系当地协会获取帮助。
Valerie Grinblade 芬德林镇广场 9 我会先离开广场，避开Valerie Grinblade和其他红袍恶棍团伙成员。然后，我会与市中心的居民互动，了解情况和寻找商机，并考虑前往Toblen Stonehill或Linene Graywind的贸易站。
    - Emoji Represent

KeyboardInterrupt: ignored